# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `clanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g., copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g., `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g., `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for some sequence containers (e.g., `vector` of the `std` namespace) and associative containers (e.g., `set`, `unordered_set` of the `std` namespace).
These template instantiations are done for various *C++* fundamental types (e.g., `int`, `unsigned long int`, `double`) and the `string` of the `std` namespace.
For ordered associative containers only the `std::less` comparator was used.
For the complete procedure refer to the `AutoWIG.py` file situed at the root of the **STL** [repository](https://github.com/StatisKit/STL).


We here aim at presenting how template libraries can be wrapped.
First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

On Windows OSes, the visual studio version used to compile future wrappers must be given.
But if the **SCons** tool is used, this version is known.

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [2]:
import sys
PYTHON_VERSION = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to detect the **Git** repository root

In [3]:
GIT_ROOT = !git rev-parse --show-toplevel
GIT_ROOT = GIT_ROOT.pop()

In this notebook, we do not need to import **AutoWIG** since **SCons** is configured to use the **Boost.Python** tool installed with **AutoWIG** that can be used to generate wrappers (see the `../git/STL/src/cpp/SConscript` file).

In [4]:
import os
SCONSCRIPT = os.path.join(GIT_ROOT, 'share', 'git', 'STL', 'src', 'cpp', 'SConscript')
!pygmentize {SCONSCRIPT}

# -*-python-*-

VECTORS = ['Index',
           'int',
           'double',
           'std::string']

SETS = { 'less': ['Index',
                  'int',
                  'double',
                  'std::string'],
         'none': ['Index']}

HEADER = """\
#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
   

The controller is registered in the `../git/STL/src/cpp/AutoWIG.py` file

In [5]:
AUTOWIG = os.path.join(GIT_ROOT, 'share', 'git', 'STL', 'src', 'cpp', 'AutoWIG.py')
!pygmentize {AUTOWIG}

import autowig
import itertools

def controller(asg, **kwargs):
    autowig.controller.plugin = 'default'
    asg = autowig.controller(asg, **kwargs)
    for function in asg['::statiskit::stl'].functions():
        if function.localname in ['generator', 'insert']:
            parameter = function.parameters[0].qualified_type.desugared_type
            if parameter.is_class:
                function.parent = parameter.unqualified_type
    for cls in asg['class ::std::vector'].specializations(partial = False):
        for method in cls.methods():
            if method.localname in ['resize', 'shrink_to_fit', 'operator[]']:
                if isinstance(method.boost_python_export, bool):
                    method.boost_python_export = False
        for constructor in cls.constructors():
            if not(constructor.nb_parameters == 0 or constructor.nb_parameters == 1 and constructor.parameters[0].qualified_type.unqualified_type == cls):
                if isinstance(constructor.boost_p

Then, in addition to the **STL** library, the **StatisKit.STL** library has to be installed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [6]:
!conda remove libstatiskit_stl -y
CONDA_RECIPE = os.path.join(GIT_ROOT, 'share', 'git', 'STL', 'bin', 'conda', 'libstatiskit_stl')
!conda build --python={PYTHON_VERSION} {CONDA_RECIPE} -c statiskit
!conda install -y libstatiskit_stl --use-local -c statiskit

Fetching package metadata .................
Solving package specifications: .

Package plan for package removal in environment /home/pfernique/.miniconda2/envs/fp17py3k:

The following packages will be REMOVED:

    libstatiskit_stl: 1.0.0-h5ddfd24_0 local

Attempting to finalize metadata for libstatiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for libstatiskit_stl
BUILD START: [u'libstatiskit_stl-1.0.0-h5ddfd24_0.tar.bz2']

The following NEW packages will be INSTALLED:

    binutils_impl_linux-64: 2.28.1-h04c84fa_2             
    binutils_linux-64:      7.2.0-25                      
    ca-certificates:        2017.08.26-h1d4fec5_0         
    certifi:                2017.11.5-py27h71e7faf_0      
    gcc_impl_linux-64:      7.2.0-hc5ce805_2              
    gcc_linux-64:           7.2.0-25                      
    gfortran_impl_linux-64: 7.2.0-hb3c8cce_2              
    gfortran_linux-64:      7.2.0-25                      
    gxx_impl_linux-64:      7.

INFO: activate-gfortran_linux-64.sh made the following environmental changes:
+DEBUG_FFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/libstatiskit_stl_1516373229946/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold/include -fdebug-prefix-map=${SRC_DIR}=/usr/local/src/conda/${PKG_NAME}-${PKG_VERSION} -fdebug-prefix-map=${PREFIX}=/usr/local/src/conda-prefix -fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-all -fno-plt -Og -g -Wall -Wextra -fcheck=all -fbacktrace -fimplicit-none -fvar-tracking-assignments -pipe
+DEBUG_FORTRANFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/libstatiskit_stl_1516373229946/_h_env_placehold_pla

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations (see the `../git/STL/src/cpp/STL.h` file).

In [7]:
TYPEDEFS = os.path.join(GIT_ROOT, 'share', 'git', 'STL', 'src', 'cpp', 'STL.h')
!pygmentize {TYPEDEFS}

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **STL** library.
To do so, we need then to install the *C++* headers. 
This is done using the `cpp` target in **SCons**.

In [8]:
REPO_DIR = os.path.join(GIT_ROOT, 'share', 'git', 'STL')
!scons cpp -C {REPO_DIR}

scons: Entering directory `/home/pfernique/Desktop/devel/FP17/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: *** [cpp] UnicodeDecodeError : 'ascii' codec can't decode byte 0x80 in position 0: ordinal not in range(128)
scons: building terminated because of errors.


Once the headers have been installed in the system, we parse headers with relevant compilation flags.
This is done using the `autowig` target in **SCons**.

In [9]:
!scons autowig -C {REPO_DIR}

scons: Entering directory `/home/pfernique/Desktop/devel/FP17/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: *** [autowig] UnicodeDecodeError : 'ascii' codec can't decode byte 0x80 in position 0: ordinal not in range(128)
scons: building terminated because of errors.


Here is the list of the generated wrappers (untracked files).

In [10]:
!git -C {REPO_DIR} status

HEAD detached at 4756312
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   src/cpp/STL.cpp
	modified:   src/cpp/STL.h

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/py/statiskit/stl/_stl.py
	src/py/wrapper/_stl.cpp
	src/py/wrapper/_stl.h
	src/py/wrapper/wrapper_107131f9768c56e794a9b0de728d1738.cpp
	src/py/wrapper/wrapper_10b14312eeb655268489cd34090870cf.cpp
	src/py/wrapper/wrapper_167afa79f8975f7f9f29c70cd82875f3.cpp
	src/py/wrapper/wrapper_21a55b6a8dfb5aebbc61516fd32c8032.cpp
	src/py/wrapper/wrapper_2e36042e6d8259f3b9994ca91a6c6708.cpp
	src/py/wrapper/wrapper_2ec52a34f3e95050841a0f79bcbaf268.cpp
	src/py/wrapper/wrapper_3b59a0980c80518c808634f7a84dc3cd.cpp
	src/py/wrapper/wrapper_40168be1357a516d854c80b86486f31d.cpp
	src/py/wrapper/wrapper_43a2a0e505405aa98c3464a24a16ac48.cpp
	src/py/wrapper/wrapper_448c20257e485ac

And here, we present the wrappers generated for the `std::vector< int >` class.

In [11]:
WRAPPER = os.path.join(GIT_ROOT, 'share', 'git', 'STL', 'src', 'py', 'wrapper',
                       'wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp')
!pygmentize {WRAPPER}

#include "_stl.h"



namespace autowig
{

    void method_decorator_bb1e0852f2ca56c094260a03787426c7(class ::std::vector< int, class ::std::allocator< int > > & instance, ::std::vector< int, class ::std::allocator< int > >::size_type  param_in_0, int param_out) { instance.at(param_in_0) = param_out; }
    void method_decorator_7ec1ac72b0b05f3a9707175bcd5da0bd(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.front() = param_out; }
    void method_decorator_ed1cf37568ed54cbbd326e6ccbe5f27d(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.back() = param_out; }
}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::std::vector< int, class ::std::allocator< int > > const volatile * get_pointer<class ::std::vector< int, class ::std::allocator< int > > const volatile >(class ::std::vector< int, class ::std::allocator< int > > const volatile *c) { return c; }
}
 

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [12]:
!conda remove python-statiskit_stl -y
CONDA_RECIPE = os.path.join(GIT_ROOT, 'share', 'git', 'STL', 'bin', 'conda', 'python-statiskit_stl')
!conda build --python={PYTHON_VERSION} {CONDA_RECIPE} -c statiskit
!conda install -y python-statiskit_stl --use-local -c statiskit

Fetching package metadata .................
Solving package specifications: .

PackageNotFoundError: Package(s) is missing from the environment:
            [u'python-statiskit_stl
            

Attempting to finalize metadata for python-statiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for python-statiskit_stl
BUILD START: [u'python-statiskit_stl-3.3.1-py36h68e8a98_0.tar.bz2']

The following NEW packages will be INSTALLED:

    alabaster:                0.7.10-py36h306e16b_0         
    asn1crypto:               0.24.0-py36_0                 
    babel:                    2.5.1-py36_0                  
    binutils_impl_linux-64:   2.28.1-h04c84fa_2             
    binutils_linux-64:        7.2.0-25                      
    bzip2:                    1.0.6-h6d464ef_2              
    ca-certificates:          2017.08.26-h1d4fec5_0         
    certifi:                  2017.11.5-py36hf29ccca_0      
    cffi:                     1.11.2-py36h2825082_0         


INFO: activate-binutils_linux-64.sh made the following environmental changes:
+ADDR2LINE=/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516373261254/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-addr2line
+AR=/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516373261254/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-ar
+AS=/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516373261254/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-as
+CXXFILT=/home/pferniqu


scons py --prefix=$PREFIX -j$CPU_COUNT
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: *** [build/src/py/wrapper/_stl.os] UnicodeDecodeError : 'ascii' codec can't decode byte 0x80 in position 0: ordinal not in range(128)
scons: building terminated because of errors.
Traceback (most recent call last):
  File "/home/pfernique/.miniconda2/bin/conda-build", line 11, in <module>
    sys.exit(main())
  File "/home/pfernique/.miniconda2/lib/python2.7/site-packages/conda_build/cli/main_build.py", line 388, in main
    execute(sys.argv[1:])
  File "/home/pfernique/.miniconda2/lib/python2.7/site-packages/conda_build/cli/main_build.py", line 379, in execute
    verify=args.verify)
  File "/home/pfernique/.miniconda2/lib/python2.7/site-packages/conda_build/api.py", line 187, in build
    need_source_download=need_source_download, config=config, variants=variants)
  File "/home/pfernique/.miniconda2/lib/python2.7/site-packages/conda_buil

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [13]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

ModuleNotFoundError: No module named 'statiskit'

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
VectorInt([0, 1])

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SITE_SCONS'],
                       'site_autowig',
                       'ASG',
                       'statiskit_stl.pkl'), 'rb') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)